# Структуры

**Структура** - это тип данных, который позволяет объединить в себе 0 или больше именованных значений разных типов. Эти значения называются **полями**(field). Пример:

In [1]:
import (
    "fmt"
    "time"
)

// Объявление структуры Employee.
type Employee struct {
    ID        int
    Name      string
    Address   string
    DoB       time.Time
    Position  string
    Salary    int
    ManagerID int
}

// Создание экземпляра Employee.
var dilbert Employee

Обратиться к отдельным полям можно через "." как, например, dilbert.Name. Т.к. dilbert является переменной, её поля тоже являются переменными, т.е. их можно менять также как и получать адрес:

In [2]:
dilbert.Salary -= 1000
fmt.Println(dilbert.Salary)

position := &dilbert.Position
*position += "Senior " + *position
fmt.Sprint(dilbert.Position)

-1000
Senior 


Точку также используют для доступа к полям **даже** в случае работы с указателем на структуру:

In [3]:
var hardWorker *Employee = &dilbert
// Следующее выражение эквивалентно (*hardWorker).Position += " (proactive team player)".
hardWorker.Position += " (proactive team player)"

fmt.Sprint(dilbert.Position)

Senior  (proactive team player)


Поля структуры обычно записывают каждый на своей строке, но последовательные поля одного типа можно объявить вместе:

In [4]:
type Employee struct {
    ID            int
    Name, Address string
    DoB           time.Time
    Position      string
    Salary        int
    ManagerID     int
}

Имя поля структуры **экспортируется**, если оно начинается с большой буквы. Это основной механизм котроля доступа, используемый в Go.

Структура с именем S **не может** создать внутри себя поле типа S: агрегированные значения не могут содержать сами себя(аналогичное ограничение применимо и к массивам). Но структура S может объявить **поле-указатель**, которое является указателем на S. Таким образом можно создавать рекурсивные структуры данных как связный лист или дерево:

In [5]:
type Node struct {
    value int
    next  *Node
}

var head Node = Node{1, nil}
head.next = &Node{2, nil}

fmt.Sprint(head.value, " -> ", head.next.value)

1 -> 2


Нулевое значение структуры формируется из нулевых значений его полей.

**struct{}** называют пустой структурой, ей размер равен 0 и она не хранит какой либо полезной информации.

## Литерал структуры

Значение типа структуры можно создать с помощью **литерала структуры**, который определяет значения его полей. Например:

In [6]:
type Point struct {X, Y int}

p := Point{1, 2}
fmt.Sprint(p.X, ", ", p.Y)

1, 2


Существует **2 вида** литералов структур: при использовании **первого** вида необходимо указывать значение каждого поля именно в том виде, в котором поля объявлены в структуре. Именно такой литерал был использован в примере выше.  
Гораздо чаще используется **второй** вид литералов, когда поля инициализируются перечислением названия некоторых или всех полей структуры и их значений. Если в таком литерале какое-либо поле пропущено, то для неё устанавливается нулевое значение её типа. Т.к. мы явно указываем название поля, порядок не имеет значения. Пример:

In [7]:
q := Point{Y: -1, X: 8}
fmt.Sprint(q.X, ", ", q.Y)

8, -1
